In [ ]:
import numpy as np
import torch

In [ ]:
# set logger and enforce reproducibility
from GPErks.log.logger import get_logger
from GPErks.utils.random import set_seed
log = get_logger()
seed = 8
set_seed(seed)  # reproducible sampling

**2D function example**: Currin et al. (1988)

$f(x) = \left[1 - \exp{\left(1 - \dfrac{1}{2 x_2}\right)}\right]\,\left(\dfrac{2300 x_{1}^3 + 1900 x_{1}^2 + 2092 x_{1} + 60}{100 x_{1}^3 + 500 x_{1}^2 + 4 x_{1} + 20}\right)$

In [ ]:
# function to learn (normally a high-dimensional, expensive deterministic model)
from GPErks.utils.test_functions import currin_exp
f = lambda X: np.array([currin_exp(x) for x in X])
D = 2

In [ ]:
# build dataset
from GPErks.gp.data.dataset import Dataset
dataset = Dataset.build_from_function(
    f,
    D,
    n_train_samples=20,
    n_test_samples=25,
    design="lhs",
    seed=seed,
)

In [ ]:
# choose likelihood
from gpytorch.likelihoods import GaussianLikelihood
likelihood = GaussianLikelihood()

In [ ]:
# choose mean function
from gpytorch.means import LinearMean
mean_function = LinearMean(input_size=dataset.input_size)

In [ ]:
# choose kernel
from gpytorch.kernels import RBFKernel, ScaleKernel
kernel = ScaleKernel(RBFKernel(ard_num_dims=dataset.input_size))

In [ ]:
# choose metrics
from torchmetrics import MeanSquaredError, R2Score
metrics = [MeanSquaredError(), R2Score()]

In [ ]:
# define experiment
from GPErks.gp.experiment import GPExperiment
experiment = GPExperiment(
    dataset,
    likelihood,
    mean_function,
    kernel,
    n_restarts=3,
    metrics=metrics,
    seed=seed  # reproducible training
)

In [ ]:
# dump experiment in config file
import os
notebook_full_path = os.path.join(os.getcwd(), nb_name)
file = os.path.basename(notebook_full_path)
config_file = file.replace(".ipynb", ".ini")
experiment.save_to_config_file(config_file)

In [ ]:
# load experiment from config file
from GPErks.gp.experiment import load_experiment_from_config_file
experiment = load_experiment_from_config_file(
    config_file,
    dataset
)

In [ ]:
# choose training options: device + optimizer
device = "cuda" if torch.cuda.is_available() else "cpu"
optimizer = torch.optim.Adam(experiment.model.parameters(), lr=0.1)

In [ ]:
# train model
from GPErks.train.emulator import GPEmulator
emulator = GPEmulator(experiment_2, device)

In [ ]:
# snapshotting
from GPErks.serialization.path import posix_path
from GPErks.train.snapshot import EveryEpochSnapshottingCriterion

snapshot_dir = posix_path(".", "snapshot", "pippo")
train_restart_template = "restart_{restart}"
train_epoch_template = "epoch_{epoch}.pth"

snpc = EveryEpochSnapshottingCriterion(
    posix_path(
        snapshot_dir,
        train_restart_template,
    ),
    train_epoch_template
)

# training
emulator.train(optimizer, snapshotting_criterion=snpc)

In [ ]:
# inference on stored test set
from GPErks.perks.inference import Inference
inference = Inference(emulator)
inference.summary()

In [ ]:
# loading emulator
from pathlib import Path
# best_model_file = posix_path(".",
#     Path(snpc.snapshot_dir).parent.as_posix(),
#     "best_model.pth"
# )  # for some reason, symbolic link not working on a notebook

best_restart = 1
best_epoch = 100
best_model_file = snpc.get_snapshot_file_path(best_restart, best_epoch)

emulator_2 = GPEmulator(experiment_2, device)
emulator_2.model = torch.load(best_model_file, map_location=torch.device(device))

In [ ]:
inference = Inference(emulator)
inference.summary()